<a href="https://colab.research.google.com/github/noahkarsky/remote-sensing-work/blob/main/Reunion_Island.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geemap
!pip install eemont

     |████████████████████████████████| 476 kB 8.2 MB/s 
     |████████████████████████████████| 98 kB 7.1 MB/s 
     |████████████████████████████████| 3.3 MB 42.6 MB/s 
     |████████████████████████████████| 130 kB 44.6 MB/s 
     |████████████████████████████████| 1.2 MB 38.6 MB/s 
     |████████████████████████████████| 76 kB 1.0 MB/s 
     |████████████████████████████████| 76 kB 4.2 MB/s 
     |████████████████████████████████| 219 kB 42.1 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████| 1.3 MB 35.8 MB/s 
     |████████████████████████████████| 1.6 MB 48.8 MB/s 
     |████████████████████████████████| 115 kB 60.8 MB/s 
     |████████████████████████████████| 112 kB 60.4 MB/s 
     |████████████████████████████████| 392 kB 43.4 MB/s 
     |████████████████████████████████| 551 kB 33.6 MB/s 
     |████████████████████████████████| 68 kB 6.6 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
     |█████████████████████

     |████████████████████████████████| 115 kB 7.7 MB/s 
  Created wheel for eemont: filename=eemont-0.2.5-py3-none-any.whl size=112562 sha256=c6bab26823e637b4698ecea0640913081a9be20bb6cbaf94efceb050a2e35527
  Stored in directory: /root/.cache/pip/wheels/6b/59/5a/4adbba5c684d32bcbfb75af879f499a56738f65ba567aa7f94
Successfully built eemont


In [2]:
import ee
import geemap
import eemont
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image
import geemap.colormaps as cm
%matplotlib inline

In [3]:
#authenticate and initialize ee
Map = geemap.Map()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=XuNPdWQ9RuYqIDhfKgqo-Lc5AR4Nw44GdXIpT0tl1nE&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiglD86E7uE0BNVyn1vuZTHPq29lUH3AuU1cxhdWFq0921OyokvdEY

Successfully saved authorization token.


In [35]:
#rectangle drawn at Reunion Island
AOI = ee.Geometry.Polygon([
          [
            [
              52.6025390625,
              -25.18505888358066
            ],
            [
              58.46923828125001,
              -25.18505888358066
            ],
            [
              58.46923828125001,
              -21.145992164957878
            ],
            [
              52.6025390625,
              -21.145992164957878
            ],
            [
              52.6025390625,
              -25.18505888358066
            ]
          ]
        ])

aoi_point = AOI.centroid()
lng, lat = aoi_point.getInfo()['coordinates']
print("lng = {}, lat = {}".format(lng, lat))

lng = 55.535888671874986, lat = -23.18248328293058


In [66]:
#first we grab a specific image collection
sentinel_collection = ee.ImageCollection("COPERNICUS/S2_SR")

#we can also grab a filtered image collection using the poly we made above
sentinel_collection = sentinel_collection.filterBounds(AOI).filterDate("2019-10-10","2019-10-15").filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than',90)

#lets see how many images are in this collection
print(sentinel_collection.size().getInfo())

8


In [67]:


sentinel_collection.aggregate_array('system:index').getInfo()

['20191011T062449_20191011T062449_T40KFB',
 '20191014T063509_20191014T063510_T39KYS',
 '20191014T063509_20191014T063510_T39KZS',
 '20191014T063509_20191014T063510_T40KBB',
 '20191014T063509_20191014T063510_T40KCA',
 '20191014T063509_20191014T063510_T40KCB',
 '20191014T063509_20191014T063510_T40KDA',
 '20191014T063509_20191014T063510_T40KDB']

In [69]:


collectionList = sentinel_collection.toList(sentinel_collection.size())

In [70]:
ndwi_vis = {
    "min": 0,
    "max": 1,
    "palette": cm.palettes.ndwi,
    'dimensions': 512,

}

true_color_vis = {'min': 0,
              'max': 3000,
              'dimensions': 1024,
              'bands': ['B4', 'B3', 'B2']
              }

#calculate the ndwi
ndwi = ee.Image(collectionList.get(0)).normalizedDifference(['B3','B8'])
true_color = ee.Image(collectionList.get(0))

Image(url = ndwi.getThumbUrl(ndwi_vis))

In [54]:
cm.palettes.ndwi

('#ece7f2',
 '#d0d1e6',
 '#a6bddb',
 '#74a9cf',
 '#3690c0',
 '#0570b0',
 '#045a8d',
 '#023858')

In [75]:
ndwi_vis = {
    "min": -1,
    "max": 0.5,
    "palette": cm.palettes.ndwi,

}

true_color_vis = {'min': 0,
              'max': 3000,
              'bands': ['B4', 'B3', 'B2']
              }


Map = geemap.Map(center=[-21.0211,58.9251], zoom=20, )
#grabbing image to add to the map
ndwi = ee.Image(collectionList.get(0)).normalizedDifference(['B3','B8'])
true_color = ee.Image(collectionList.get(0))

#adding layers to the map
# Map.addLayer(AOI,name='AOI')
Map.addLayer(ndwi,ndwi_vis, 'ndwi')
Map.addLayer(true_color,true_color_vis,'true color')
Map.addLayerControl()

Map